In [ ]:
import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")
scope.default_setup()

In [ ]:
# clocks:
if PLATFORM == 'CW305_IBEX':
    print('Set the CW305 J16 switch to 1 so that Ibex is clocked from HS2. Note this is different from what most other CW305 notebooks require.')

scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 100e6
if scope._is_husky:
    scope.clock.adc_mul = 1
else:
    scope.clock.adc_src = 'clkgen_x1'

target.baud = 115200

In [ ]:
# Ibex requires external JTAG programming
prog = None

# But let's program the FPGA bitfile:
if PLATFORM == 'CW305_IBEX':
    from chipwhisperer.hardware.firmware.cw305 import getsome
    bsdata = getsome(f"lowrisc_ibex_demo_system.bit")
    cw305 = cw.target(None, cw.targets.CW305, bsfile=None, force=False)
    status = cw305.fpga.FPGAProgram(bsdata, exceptOnDoneFailure=False, prog_speed=10e6)
    
elif PLATFORM == 'CW312_IBEX':
    from chipwhisperer.hardware.firmware.xc7a35 import getsome
    bsdata = getsome(f"lowrisc_ibex_demo_system.bit")
    #from chipwhisperer.hardware.naeusb.programmer_targetfpga import CW312T_XC7A35T
    fpga = cw.hardware.naeusb.programmer_targetfpga.CW312T_XC7A35T(scope)
    fpga.program(bsdata, sck_speed=10e6)
    status = fpga.done_state()

else:
    raise ValueError('Unsupported PLATFORM: %s' % PLATFORM)

if status:
    print("✅ FPGA programmed. Next you need to program the firmware using the load_demo_system.sh command from the Ibex repository.")
else:
    print("❌ FPGA Done pin failed to go high")

In [ ]:
def reset_target(scope):
    raise IOError("Default Ibex build does not have Python-driven reset - use the R1 button on the CW305, or the S1 button on the CW313")
